In [10]:
## headings
# reference: https://github.com/eriklindernoren/Keras-GAN/blob/master/pix2pix/data_loader.py
#            https://github.com/equinor/segyio
import os
import struct
import segyio
import numpy as np


In [20]:
class Dataloader():
    def __init__(self, data_path, nt, nr, nph):
        self.data_path = data_path
        self.nt = nt
        self.nr = nr
        self.nph = nph
    
    def load_single_batch(self, batch_size=1, is_testing=False, ratio=0.5):        
        self.n_batches = int( 151 / batch_size * ratio ) #int( len(path) / batch_size * ratio )
        x_data = np.empty((batch_size, self.nt, self.nr, self.nph)) ## b-4000-300-2, vz, vx
        y_data = np.empty((batch_size, self.nt, self.nr, 1)) ## b-4000-300-2, div, curl
        i = 0
        with segyio.open(self.data_path,'r',ignore_geometry=True) as segyfile:
            while True:
#         for i in range(self.n_batches): 
                if (i + 1) * batch_size > 151 * ratio:
                    i = 0
#                 break
                for batch_i in range(batch_size):
                    for nr_i in range(self.nr):
                        with segyio.open(self.data_path,'r',ignore_geometry=True) as segyfile:
                            segyfile.mmap()
                            y_data[batch_i,:,nr_i,0] = \
                            segyfile.trace[i*batch_size*4*self.nr + batch_i * 4 * self.nr + nr_i * 4 + 0]
                            x_data[batch_i,:,nr_i,0] = \
                            segyfile.trace[i*batch_size*4*self.nr + batch_i * 4 * self.nr + nr_i * 4 + 1]
                            x_data[batch_i,:,nr_i,1] = \
                            segyfile.trace[i*batch_size*4*self.nr + batch_i * 4 * self.nr + nr_i * 4 + 2]
                            x_data[batch_i,:,nr_i,2] = \
                            segyfile.trace[i*batch_size*4*self.nr + batch_i * 4 * self.nr + nr_i * 4 + 3]                 
                yield x_data, y_data
                i = i + 1

In [21]:
# data_path = "/media/wywdisk/VSPdata/data/haveinvx/layer2_haveinvx"
# data_path = "/home/wyw/data/MKDATA/mkvspdata/E28093/E-layer4"
data_path = '/media/wywdisk/VSPdata/data/realData/SEAM_I_Walkaway_VSP/SEAM_Well1VSP_Shots23900.sgy'
my_data_loader = Dataloader(data_path, 2001, 467, 3)

In [22]:
# my_data_loader.load_batch(batch_size=1, is_testing=False, ratio=0.001)

In [24]:
for batch_i, (x_data, y_data) in enumerate(my_data_loader.load_single_batch(batch_size=10, is_testing=False, ratio=0.5)):
    print(batch_i)
#     print(x_data.shape, y_data.shape )
#     print(x_data,y_data)
#     a1 = y_predict
#     print("Information of a1 :")
    print("The shape of a1   :",x_data.shape)
    print("The range        : ", [x_data[-1,:,:,0].min(),x_data[-1,:,:,0].max()])
    print("The range        : ", [x_data[-1,:,:,1].min(),x_data[-1,:,:,1].max()])
    print("The range        : ", [x_data[-1,:,:,2].min(),x_data[-1,:,:,2].max()])
    print("The range        : ", [y_data[-1,:,:,0].min(),y_data[-1,:,:,0].max()])

0
The shape of a1   : (10, 2001, 467, 3)
The range        :  [-43.123931884765625, 41.91886901855469]
The range        :  [-132.93368530273438, 136.0438995361328]
The range        :  [-15.594023704528809, 13.357939720153809]
The range        :  [-3055.13720703125, 4490.5234375]
1
The shape of a1   : (10, 2001, 467, 3)
The range        :  [-25.686737060546875, 22.418243408203125]
The range        :  [-128.85986328125, 121.51829528808594]
The range        :  [-9.360477447509766, 7.8704376220703125]
The range        :  [-3213.386474609375, 4803.2109375]
2
The shape of a1   : (10, 2001, 467, 3)
The range        :  [-40.97137451171875, 38.68865966796875]
The range        :  [-137.7332763671875, 116.99676513671875]
The range        :  [-9.14682388305664, 9.11376667022705]
The range        :  [-3617.08056640625, 5227.51171875]
3
The shape of a1   : (10, 2001, 467, 3)
The range        :  [-92.5177001953125, 88.19300842285156]
The range        :  [-170.25721740722656, 124.31304931640625]
The ra

KeyboardInterrupt: 

In [ ]:
def get_percentile(data, perc):
    ## data shape, [0,:,:,0]
    dd1, dd2 = np.shape(data)
    tmp = np.zeros_like(data)
    for idd2 in range(dd2):
        tmp[:,idd2] = np.percentile(data[:,idd2], 99)
    return tmp

In [ ]:
## plot y_predict
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
# ## plot y_predict
# import matplotlib as cm
# import matplotlib.pyplot as plt
# from matplotlib.ticker import MultipleLocator
lines = 2
cols = 2
forts = 12
ss1 = 400
ss2 = 500
value_limit = 1#1e-20

x_libs = [0,0, 500, 1000, 1500, 2000]#np.linspace(0,1500,4)#[0, 500, 1000, 1500]
y_libs = np.linspace(0,2,9)#[0,0.4, 0.8, 1.2, 1.6, 2]

fig, axs = plt.subplots(1,2,figsize=(16, 20), sharey=True)

# obj = 'y_predict'
# im = plt.imshow(eval(obj)[0,:,:,1], extent=[0, 512, 512,0])
# plt.colorbar(im)

# ax1 = plt.subplot(131)
ax1 = plt.subplot(lines,cols,1)
obj = 'x_data'
target_data = eval(obj)[0,:,:,0]
value_limit=np.max( target_data ) / 10
# print(obj, ": ", eval(obj).shape, "\t", [eval(obj).min(), eval(obj).max()] )
# target_data = get_percentile(eval(obj)[0,:,:,0], 99)
# im = plt.imshow(target_data, extent=[0, ss1, ss2,0],vmin=-value_limit, vmax=value_limit)
im = plt.imshow(eval(obj)[0,:,:,0], extent=[0, ss1, ss2,0],vmin=-value_limit, vmax=value_limit,cmap="gray")
plt.colorbar(im)
# plt.title("The elastic z component of VSP data", fontsize=forts);
plt.title("vertical acceleration", fontsize=forts);
plt.xlabel('depth(m)', fontsize=forts);
plt.ylabel('time(s)', fontsize=forts);
ax1.xaxis.set_major_locator(MultipleLocator(75))
ax1.xaxis.set(ticklabels=x_libs)#[0,300,600, 900, 1200, 1500]);
ax1.yaxis.set(ticklabels=y_libs)#[0,0.4, 0.8, 1.2, 1.6, 2]);
plt.setp(ax1.get_xticklabels(), fontsize=10);
# plt.tight_layout()

# ax1 = plt.subplot(131)
ax2 = plt.subplot(lines,cols,2)
obj = 'x_data'
target_data = eval(obj)[0,:,:,0]
value_limit=np.max( target_data ) / 10
# print(obj, ": ", eval(obj).shape, "\t", [eval(obj).min(), eval(obj).max()] )
im = plt.imshow(eval(obj)[0,:,:,1], extent=[0, ss1, ss2,0],vmin=-value_limit, vmax=value_limit,cmap="gray")
plt.colorbar(im)
# plt.title("The elastic z component of VSP data", fontsize=forts);
plt.title("EW acceleration", fontsize=forts);
plt.xlabel('depth(m)', fontsize=forts);
plt.ylabel('time(s)', fontsize=forts);
# ax1 = fig.add_subplot(131)
ax2.xaxis.set_major_locator(MultipleLocator(75))
ax2.xaxis.set(ticklabels=x_libs)#[0,300,600, 900, 1200, 1500]);
ax2.yaxis.set(ticklabels=y_libs)#[0,0.4, 0.8, 1.2, 1.6, 2]);
plt.setp(ax2.get_xticklabels(), fontsize=10);
# plt.tight_layout()

# ax1 = plt.subplot(131)
ax3 = plt.subplot(lines,cols,3)
obj = 'x_data'
target_data = eval(obj)[0,:,:,0]
value_limit=np.max( target_data ) / 10
# print(obj, ": ", eval(obj).shape, "\t", [eval(obj).min(), eval(obj).max()] )
im = plt.imshow(eval(obj)[0,:,:,2], extent=[0, ss1, ss2,0],vmin=-value_limit, vmax=value_limit,cmap="gray")
plt.colorbar(im)
# plt.title("The elastic z component of VSP data", fontsize=forts);
plt.title("NS acceleration", fontsize=forts);
plt.xlabel('depth(m)', fontsize=forts);
plt.ylabel('time(s)', fontsize=forts);
# ax1 = fig.add_subplot(131)
ax3.xaxis.set_major_locator(MultipleLocator(75))
ax3.xaxis.set(ticklabels=x_libs)#[0,300,600, 900, 1200, 1500]);
ax3.yaxis.set(ticklabels=y_libs)#[0,0.4, 0.8, 1.2, 1.6, 2]);
plt.setp(ax3.get_xticklabels(), fontsize=10);
# plt.tight_layout()

# ax1 = plt.subplot(131)
ax4 = plt.subplot(lines,cols,4)
obj = 'y_data'
target_data = eval(obj)[0,:,:,0]
value_limit=np.max( target_data ) / 10
# print(obj, ": ", eval(obj).shape, "\t", [eval(obj).min(), eval(obj).max()] )
im = plt.imshow(eval(obj)[0,:,:,0], extent=[0, ss1, ss2,0],vmin=-value_limit, vmax=value_limit,cmap="gray")
plt.colorbar(im)
# plt.title("The elastic z component of VSP data", fontsize=forts);
plt.title("pressure", fontsize=forts);
plt.xlabel('depth(m)', fontsize=forts);
plt.ylabel('time(s)', fontsize=forts);
# ax1 = fig.add_subplot(131)
ax4.xaxis.set_major_locator(MultipleLocator(75))
ax4.xaxis.set(ticklabels=x_libs)#[0,300,600, 900, 1200, 1500]);
ax4.yaxis.set(ticklabels=y_libs)#[0,0.4, 0.8, 1.2, 1.6, 2]);
plt.setp(ax4.get_xticklabels(), fontsize=10);
# plt.tight_layout()



In [ ]:
plt.figure(figsize=(10,10))
ss1 = 300#467*3; 
ss2 = 400#2001
obj = 'y_data'
target_data = eval(obj)[0,:,:,0]
value_limit=np.max( target_data ) * 0.06 #/ 10
# print(obj, ": ", eval(obj).shape, "\t", [eval(obj).min(), eval(obj).max()] )
im = plt.imshow(target_data, extent=[0, ss1, ss2,0],vmin=-value_limit, vmax=value_limit, cmap="gray")
plt.colorbar(im);


In [ ]:
plt.figure(figsize=(10,10))
obj = 'x_data'
target_data = eval(obj)[0,:,:,0]
value_limit=np.max( target_data ) *0.05 #/ 10
# print(obj, ": ", eval(obj).shape, "\t", [eval(obj).min(), eval(obj).max()] )
im = plt.imshow(target_data, extent=[0, ss1, ss2,0],vmin=-value_limit, vmax=value_limit, cmap="gray")
plt.colorbar(im);

